<a href="https://colab.research.google.com/github/killianb22/BI_CA/blob/main/APA_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Base Model

In [4]:
import re
import string
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff
from collections import Counter
import pandas as pd

In [5]:
twitter_data = pd.read_csv("/content/Twitter_Data.csv")

In [6]:
print(twitter_data.shape)

(162980, 2)


In [7]:
twitter_data.head()

,clean_text,category
0,when modi promised “minimum government maximum...,-1.0
1,talk all the nonsense and continue all the dra...,0.0
2,what did just say vote for modi welcome bjp t...,1.0
3,asking his supporters prefix chowkidar their n...,1.0
4,answer who among these the most powerful world...,1.0


In [8]:
print(twitter_data.isna().sum())

clean_text    4
category      7
dtype: int64


In [9]:
twitter_data = twitter_data.dropna()

In [10]:
# Find the number of positive and negative reviews
print('Number of positive and negative reviews: ', twitter_data.category.value_counts())

# Find the proportion of positive and negative reviews
print('Proportion of positive and negative reviews: ', twitter_data.category.value_counts() / len(twitter_data))

Number of positive and negative reviews:   1.0    72249
 0.0    55211
-1.0    35509
Name: category, dtype: int64
Proportion of positive and negative reviews:   1.0    0.443330
 0.0    0.338782
-1.0    0.217888
Name: category, dtype: float64


In [11]:
# Define a dictionary for the six lexical fields related to inflation in French, English, and Turkish
lexical_fields = {
    'inflation_economics': ['inflation', 'hyperinflation', 'deflation', 'stagflation', 'price index',
                            'monetary policy', 'purchasing power', 'enflasyon', 'devalüasyon',
                            'fiyat endeksi', 'para politikası', 'alım gücü', 'inflation', 'déflation',
                            'pouvoir d\'achat', 'politique monétaire', 'indice des prix'],
    'expensive': ['expensive', 'high-priced', 'costly', 'overpriced', 'pricey',
                  'pahalı', 'maliyetli', 'yüksek fiyatlı', 'cher', 'coûteux', 'hors de prix'],
    'cheap': ['cheap', 'low-cost', 'budget', 'affordable', 'inexpensive',
              'ucuz', 'düşük maliyetli', 'ekonomik', 'pas cher', 'abordable', 'bon marché'],
    'prices_costs': ['price', 'cost', 'expense', 'fee', 'charge', 'rate', 'tariff',
                     'fiyat', 'maliyet', 'ücret', 'tarife', 'prix', 'coût', 'frais', 'tarif'],
    'statistical_institutions': ['Bureau of Labor Statistics', 'Consumer Price Index', 'Federal Reserve',
                                  'Eurostat', 'TÜİK', 'Merkez Bankası', 'İstatistik Kurumu',
                                  'Insee', 'Banque centrale', 'Office de statistique'],
    'additional_keywords': ['economy', 'market', 'value', 'money', 'finance',
                            'ekonomi', 'piyasa', 'değer', 'para', 'finans',
                            'économie', 'marché', 'valeur', 'argent', 'finance']
}

In [12]:
# Function to check if a tweet contains any of the keywords from the lexical fields
def filter_tweets(tweet, fields):
    if isinstance(tweet, str):
        return any(any(keyword in tweet.lower() for keyword in field) for field in fields.values())
    else:
        # If tweet is not a string, we do not consider it for our lexical fields
        return False

In [13]:
import re
# Apply the adjusted filter function to the DataFrame
filtered_df = twitter_data[twitter_data['clean_text'].apply(lambda tweet: filter_tweets(tweet, lexical_fields))]

# Function to identify invalid characters
# We define invalid characters as anything that is NOT alphanumeric, standard punctuation, or whitespace
def contains_invalid_characters(text):
    # Define a set of allowed punctuation characters
    allowed_punctuation = "!\"#$%&'()*+,-./:;<=>?@[\\]^_`{|}~"
    pattern = r'^[A-Za-z0-9\s' + re.escape(allowed_punctuation) + r']+$'
    return not re.match(pattern, text)

# Apply the function to the 'clean_text' column to create a mask for rows with invalid characters
invalid_mask = filtered_df['clean_text'].apply(lambda x: contains_invalid_characters(str(x)))

In [14]:
# Invert the mask to filter out the invalid rows
clean_df = filtered_df[~invalid_mask]

# Now 'clean_df' contains only rows with valid characters
print(f"Number of rows after removing invalid characters: {len(clean_df)}")

Number of rows after removing invalid characters: 14551


In [15]:
# Check for NaN values in the target variable
print("NaNs in category (before split):", clean_df['category'].isnull().sum())

NaNs in category (before split): 0


In [16]:
# Here we drop rows with NaN in 'category'
clean_df = clean_df.dropna(subset=['category'])

In [17]:
# Install the library and functions
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [18]:
import nltk, re
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize


stop = stopwords.words('english')
excluding = ['against', 'not', 'don', "don't",'ain', 'aren', "aren't", 'couldn', "couldn't",
             'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't",
             'haven', "haven't", 'isn', "isn't", 'mightn', "mightn't", 'mustn', "mustn't",
             'needn', "needn't",'shouldn', "shouldn't", 'wasn', "wasn't", 'weren',
             "weren't", 'won', "won't", 'wouldn', "wouldn't"]


stop_words = [word for word in stop if word not in excluding]
snow = SnowballStemmer('english')

def process_text(texts):
    final_text_list=[]
    for sent in texts:
        if isinstance(sent, str) == False:
            sent = ""

        filtered_sentence=[]

        sent = sent.lower()
        sent = sent.strip()
        sent = re.sub('\s+', ' ', sent)
        sent = re.compile('<.*?>').sub('', sent)

        for w in word_tokenize(sent):
            if(not w.isnumeric()) and (len(w)>2) and (w not in stop_words):
                filtered_sentence.append(snow.stem(w))
        final_string = " ".join(filtered_sentence)
        final_text_list.append(final_string)

    return final_text_list

In [19]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(clean_df[["clean_text"]],
                                                  clean_df["category"],
                                                  test_size=0.10,
                                                  shuffle=True,
                                                  random_state=324
                                                 )

In [20]:
print("Processing the clean_text fields")
X_train["clean_text"] = process_text(X_train["clean_text"].tolist())
X_test["clean_text"] = process_text(X_test["clean_text"].tolist())

Processing the clean_text fields


In [21]:
# Display the number of tweets before and after filtering/cleaning
original_count = twitter_data.shape[0]
filtered_count = clean_df.shape[0]
print(f'Original tweet count: {original_count}')
print(f'Filtered tweet count: {filtered_count}')
print(clean_df.head())

Original tweet count: 162969
Filtered tweet count: 14551
                                           clean_text  category
25  \ndevelopment has become mass movement under m...       1.0
32  modi govts slashing indias education budget cl...      -1.0
36  check out latest article premier archery leagu...       1.0
43  yes good job highly insensitivearrogant incomp...       1.0
48  with welfare delivery gst ibc and feo place mo...       1.0


In [22]:
# Displaying the number of entries in the training and testing sets
print(f"Number of entries in the training set: {len(X_train)}")
print(f"Number of entries in the testing set: {len(X_test)}")

Number of entries in the training set: 13095
Number of entries in the testing set: 1456


In [23]:
# Selecting text features for the model
text_features = ['clean_text']

# Defining the target variable for the model
model_target = 'isPositive'

In [24]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
import gensim
from gensim.models import Word2Vec
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

text_processor_0 = Pipeline([
    ('text_vect_0', CountVectorizer(binary=True, max_features=150))
])

data_preprocessor = ColumnTransformer([
    ('text_pre_0', text_processor_0, text_features[0])
])

#Increasing c value will lead to lower bias but higher variance in the model (overfitting)
pipeline = Pipeline([
    ('data_preprocessing', data_preprocessor),
    ('logistic_regression', LogisticRegression(penalty = 'l2',
                              C = 0.1))
])

# Visualizing the Pipeline
from sklearn import set_config
set_config(display='diagram')
pipeline

Pipeline(steps=[('data_preprocessing',
                 ColumnTransformer(transformers=[('text_pre_0',
                                                  Pipeline(steps=[('text_vect_0',
                                                                   CountVectorizer(binary=True,
                                                                                   max_features=150))]),
                                                  'clean_text')])),
                ('logistic_regression', LogisticRegression(C=0.1))])

In [25]:
class_labels = np.unique(y_train)

class_weights = compute_class_weight('balanced', classes=class_labels, y=y_train)

# Creating a dictionary to pass to the model
class_weights_dict = {class_labels[i]: class_weights[i] for i in range(len(class_labels))}

# Assembling the Full Pipeline
pipeline = Pipeline([
    ('data_preprocessing', data_preprocessor),
    ('logistic_regression', LogisticRegression(penalty = 'l2', class_weight=class_weights_dict,
                              C = 0.1, max_iter=1000, solver='saga'))
])

# Visualizing the Pipeline
from sklearn import set_config
set_config(display='diagram')
pipeline


Pipeline(steps=[('data_preprocessing',
                 ColumnTransformer(transformers=[('text_pre_0',
                                                  Pipeline(steps=[('text_vect_0',
                                                                   CountVectorizer(binary=True,
                                                                                   max_features=150))]),
                                                  'clean_text')])),
                ('logistic_regression',
                 LogisticRegression(C=0.1,
                                    class_weight={-1.0: 1.0338702036949312,
                                                  0.0: 1.6964632724446171,
                                                  1.0: 0.6928571428571428},
                                    max_iter=1000, solver='saga'))])

In [26]:
# Fit the Pipeline to training data
pipeline.fit(X_train, y_train.values)

Pipeline(steps=[('data_preprocessing',
                 ColumnTransformer(transformers=[('text_pre_0',
                                                  Pipeline(steps=[('text_vect_0',
                                                                   CountVectorizer(binary=True,
                                                                                   max_features=150))]),
                                                  'clean_text')])),
                ('logistic_regression',
                 LogisticRegression(C=0.1,
                                    class_weight={-1.0: 1.0338702036949312,
                                                  0.0: 1.6964632724446171,
                                                  1.0: 0.6928571428571428},
                                    max_iter=1000, solver='saga'))])

In [27]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

# Predict the categories of the testing set tweets
test_predictions = pipeline.predict(X_test)

# Printing the Confusion Matrix
print(confusion_matrix(y_test.values, test_predictions))

# Printing the Classification Report
print(classification_report(y_test.values, test_predictions))

# Calculating and Printing the Accuracy
print("Accuracy (validation):", accuracy_score(y_test.values, test_predictions))
#report = classification_report(y_train, test_predictions, output_dict=False)
#print(report)

[[196 173  83]
 [ 35 244  20]
 [119 239 347]]
              precision    recall  f1-score   support

        -1.0       0.56      0.43      0.49       452
         0.0       0.37      0.82      0.51       299
         1.0       0.77      0.49      0.60       705

    accuracy                           0.54      1456
   macro avg       0.57      0.58      0.53      1456
weighted avg       0.62      0.54      0.55      1456

Accuracy (validation): 0.540521978021978


# Improving model

The model achieved a poor initial value of 58.2% accuracy, with field '0' performing the worst with 37% precision score and 48% f1-score the model will have to be tuned in order to compute a higher score

In [28]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

# Define the KNN pipeline
pipeline = Pipeline([
    ('text_vect', CountVectorizer(binary=True, max_features=200)),  # Convert text to vector
    ('knn', KNeighborsClassifier(n_neighbors=5))  # KNN classifier with default hyperparameters
])